### Simple motion play of `Common Rig` 

In [1]:
import pickle
import mujoco
import numpy as np
import matplotlib.pyplot as plt
from mujoco_parser import MuJoCoParserClass
from util import rpy2r
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.6]


### Parse `scene_common_rig.xml`

In [2]:
xml_path = '../asset/common_rig/scene_common_rig_v2.xml'
env = MuJoCoParserClass(name='Common Rig',rel_xml_path=xml_path,VERBOSE=True)
# Modify the color of body exluding 'world'
for body_name in env.body_names:
    if body_name in ['world']: continue
    body_idx = env.body_names.index(body_name)
    geom_idxs = [idx for idx,val in enumerate(env.model.geom_bodyid) if val==body_idx]
    for geom_idx in geom_idxs:
        env.model.geom(geom_idx).rgba = [0.3,0.3,0.5,0.5]
print ("Done.")

dt:[0.0050] HZ:[200]
n_dof (=nv):[41]
n_geom:[26]
geom_names:['floor', 'base', 'root2spine', 'spine2neck', 'neck2rshoulder', 'neck2lshoulder', 'rshoulder2relbow', 'relbow2rwrist', 'rthumb', 'rpalm', 'lshoulder2lelbow', 'lelbow2lwrist', 'lthumb', 'lpalm', 'head', 'nose', 'base2rpelvis', 'rpelvis2rknee', 'rknee2rankle', 'rankle', 'rfoot', 'base2lpelvis', 'lpelvis2lknee', 'lknee2lankle', 'lankle', 'lfoot']
n_body:[20]
body_names:['world', 'base', 'torso', 'spine', 'neck', 'right_shoulder', 'right_elbow', 'right_hand', 'left_shoulder', 'left_elbow', 'left_hand', 'head', 'right_leg', 'right_pelvis', 'right_knee', 'right_ankle', 'left_leg', 'left_pelvis', 'left_knee', 'left_ankle']
n_joint:[36]
joint_names:['base', 'root1', 'root2', 'root3', 'spine', 'rs1', 'rs2', 'rs3', 're', 'rw1', 'rw2', 'rw3', 'ls1', 'ls2', 'ls3', 'le', 'lw1', 'lw2', 'lw3', 'head1', 'head2', 'head3', 'rp1', 'rp2', 'rp3', 'rk', 'ra1', 'ra2', 'ra3', 'lp1', 'lp2', 'lp3', 'lk', 'la1', 'la2', 'la3']
joint_types:[0 3 3 3 3 3 3

In [3]:
# Set which joints to control
rev_joint_names = env.ctrl_names # <==
joint_idxs_fwd = env.get_idxs_fwd(joint_names=rev_joint_names)
joint_idxs_jac = env.get_idxs_jac(joint_names=rev_joint_names)
q_rev_init = env.get_qpos_joints(rev_joint_names)
n_rev_joint = len(rev_joint_names)

### Load motion and playback

In [4]:
pkl_path = '../data/VAAI_Non_M_01_de_01_results.pkl'
with open(pkl_path,'rb') as f: data = pickle.load(f)
sample_qs = data['sample_rot'].transpose(0,2,1) # q: [5 x 198 x 35]
sample_p_roots = data['sample_trs'].transpose(0,2,1) # q: [5 x 198 x 3]
L = data['motion_length']
print ("L:[%d]"%(L))

# Initialize MuJoCo viewer
env.init_viewer(viewer_title='Common Rig',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,0.8])
env.reset()
tick = 0
while env.is_viewer_alive():
    
    # Update 
    q = sample_qs[0,tick,:] # [35]
    p_root = sample_p_roots[0,tick,:] # [3]
    # tick = min(tick+1,L-1)
    if tick==(L-1): tick = 0
    else: tick = tick + 1
    env.set_p_root(root_name='base',p=p_root)
    env.forward(q=q,joint_idxs=joint_idxs_fwd)
    
    if env.loop_every(tick_every=1):
        # Plot world frame
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label="tick:[%d]"%(tick))
        # Plot foot
        env.plot_geom_T(geom_name='rfoot',axis_len=0.3)
        env.plot_geom_T(geom_name='lfoot',axis_len=0.3)
        # Plot revolute joints with arrow
        env.plot_joint_axis(axis_len=0.1,axis_r=0.01)    
        env.render()
# Close MuJoCo viewer
env.close_viewer()
print ("Done.")

L:[198]
Pressed ESC
Quitting.
Done.


### First, get the sequence of two feet pose

In [5]:
p_rfoot_traj = np.zeros((L,3))
p_lfoot_traj = np.zeros((L,3))
for tick in range(L):
    q = sample_qs[0,tick,:] # [35]
    p_root = sample_p_roots[0,tick,:] # [3]
    env.set_p_root(root_name='base',p=p_root)
    env.forward(q=q,joint_idxs=joint_idxs_fwd)
    # Append
    p_rfoot_traj[tick,:] = env.get_p_geom(geom_name='rfoot')
    p_lfoot_traj[tick,:] = env.get_p_geom(geom_name='lfoot')

In [6]:
# Initialize MuJoCo viewer
env.init_viewer(viewer_title='Common Rig',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,0.8])
env.reset()
tick = 0
p_rfoot_centered_traj = np.zeros((L,3))
p_lfoot_centered_traj = np.zeros((L,3))
while env.is_viewer_alive():
    
    # Update
    q = sample_qs[0,tick,:] # [35]
    p_root = sample_p_roots[0,tick,:] # [3]
    p_cfoot = 0.5*(p_rfoot_traj[tick,:]+p_lfoot_traj[tick,:])
    env.set_p_root(root_name='base',p=p_root-p_cfoot+np.array([0,0,0.02]))
    env.forward(q=q,joint_idxs=joint_idxs_fwd)
    
    # Append translated foot traj
    p_rfoot_centered_traj[tick,:] = env.get_p_geom(geom_name='rfoot')
    p_lfoot_centered_traj[tick,:] = env.get_p_geom(geom_name='lfoot')
    
    # Render
    if env.loop_every(tick_every=1):
        # Plot world frame
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label="tick:[%d]"%(tick))
        # Plot foot
        env.plot_geom_T(geom_name='rfoot',axis_len=0.3)
        env.plot_geom_T(geom_name='lfoot',axis_len=0.3)
        # Plot revolute joints with arrow
        env.plot_joint_axis(axis_len=0.1,axis_r=0.01)    
        env.render()
        
    # Increase tick
    if tick==(L-1): tick = 0
    else: tick = tick + 1
# Close MuJoCo viewer
env.close_viewer()
print ("Done.")

Pressed ESC
Quitting.
Done.


### Solve IK for two feet

In [15]:
p_trgt_rfoot = np.average(p_rfoot_centered_traj,axis=0)
p_trgt_lfoot = np.average(p_lfoot_centered_traj,axis=0)
R_trgt_rfoot = rpy2r(np.radians([0,0,0]))
R_trgt_lfoot = rpy2r(np.radians([0,0,0]))
# Initialize MuJoCo viewer
env.init_viewer(viewer_title='Common Rig',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,0.8])
env.reset()
while env.is_viewer_alive():
    # Update
    q = sample_qs[0,tick,:] # [35]
    p_root = sample_p_roots[0,tick,:] # [3]
    p_cfoot = 0.5*(p_rfoot_traj[tick,:]+p_lfoot_traj[tick,:])
    env.set_p_root(root_name='base',p=p_root-p_cfoot+np.array([0,0,0.02]))
    env.forward(q=q,joint_idxs=joint_idxs_fwd)
    
    # Solve IK
    ik_geom_names = ['rfoot','lfoot']
    ik_p_trgts = [p_trgt_rfoot,p_trgt_lfoot]
    ik_R_trgts = [R_trgt_rfoot,R_trgt_lfoot]
    err_traj = np.zeros(30)
    for ik_tick in range(30):
        J_list,ik_err_list = [],[]
        for ik_idx,ik_geom_name in enumerate(ik_geom_names):
            ik_p_trgt = ik_p_trgts[ik_idx]
            ik_R_trgt = ik_R_trgts[ik_idx]
            J,ik_err = env.get_ik_ingredients_geom(
                geom_name=ik_geom_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
                IK_P=True,IK_R=True)
            J_list.append(J)
            ik_err_list.append(ik_err)
        J_stack      = np.vstack(J_list)
        ik_err_stack = np.hstack(ik_err_list)
        err_traj[ik_tick] = np.max(np.abs(ik_err_stack))
        dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
        q = q + dq[joint_idxs_jac]
        env.forward(q=q,joint_idxs=joint_idxs_fwd)
        
    # Render
    if env.loop_every(tick_every=1):
        # Plot world frame
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label="tick:[%d]"%(tick))
        # Plot foot
        env.plot_geom_T(geom_name='rfoot',axis_len=0.3)
        env.plot_geom_T(geom_name='lfoot',axis_len=0.3)
        # Plot revolute joints with arrow
        env.plot_joint_axis(axis_len=0.1,axis_r=0.01)    
        env.render()
        
    # Increase tick
    if tick==(L-1): tick = 0
    else: tick = tick + 1
# Close MuJoCo viewer
env.close_viewer()
print ("Done.")

Pressed ESC
Quitting.
Done.
